### Модуль предобработки данных. 

In [1]:
#pip install scikit-learn==1.1
#%pip install --upgrade scikit-learn==1.4.0

In [19]:
import pandas as pd
import os
import pickle

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

##### Имитируем подачу данных с датчиков

In [34]:
# train_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD001.csv")
# train_FD002 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD002.csv")
# train_FD003 = pd.read_csv("C:/Users\Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD003.csv")
# train_FD004 = pd.read_csv("C:/Users\Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD004.csv")

# test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD001.csv")
# test_FD002 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD002.csv")
# test_FD003 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD003.csv")
# test_FD004 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD004.csv")

#This dataset using for show preprocess handling incoming data
#BAD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/BAD_test_FD001.csv")

Small_BAD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/Small_BAD_test_FD001.csv")
Small_GOOD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/Small_GOOD_test_FD001.csv")

##### Проверим "плохой" датасет

In [35]:
Small_GOOD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1,0.0023,0.0003,100.0
1,1,2,-0.0027,-0.0003,100.0
2,1,3,0.0003,0.0001,100.0


In [36]:
Small_BAD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1.0,0.0023,NaN,NaN
1,1,NaN,-0.0027,-0.0003,100.0
2,1,3.0,NaN,0.0001,100.0


In [ ]:
null_set = pd.Dadataframe(Small_test_FD001.isnull())
null_set

Поиск пропусков

In [ ]:
def is_null(dataframe: pd.DataFrame):
    count_nulls = 0
    null_set = pd.Dadataframe(dataframe.isnull())

    for col in dataframe.columns:
        for val in dataframe[col].is_null():
            if (val == True):
                count_nulls +=1
            #print(val)
        print("in ", col, "nulls: ", count_nulls)
        count_nulls = 0

is_null(Small_test_FD001)

##### Создаем SimpleInputer - простой заполнитель пропущенных значений в датасете

Так как предполагается, что данные упорядоченны по циклам работы двигателя, не целесообразно использовать стандартный заполнитель SimpleImputer. разумнее будет вычислить среднее значение лишь до и после пропущенного значения в каждом столбце. Можно использовать KNNImputer.

In [37]:
simple_inputer = KNNImputer(n_neighbors = 2)
std_scaler = StandardScaler()

pipe_num = Pipeline([('imputer', simple_inputer),('scaler', std_scaler)])

Обучаем Pipeline

In [43]:
res_num = pipe_num.fit_transform(Small_GOOD_test_FD001)

Теперь можем применить полученный pipeline
Видим, что пропусков нет

In [44]:
res_num = pd.DataFrame(res_num, columns=pipe_num['scaler'].get_feature_names_out(Small_GOOD_test_FD001.columns))
res_num

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,0.0,-1.224745,1.135550,1.069045,0.0
1,0.0,0.000000,-1.297771,-1.336306,0.0
2,0.0,1.224745,0.162221,0.267261,0.0


In [41]:
Small_GOOD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1,0.0023,0.0003,100.0
1,1,2,-0.0027,-0.0003,100.0
2,1,3,0.0003,0.0001,100.0


Сохраняем Pipline

In [18]:
savePath = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/final/Pipelinr.pickle'

with open(savePath, 'wb') as handle:
    save_pik_pipeline = pickle.dumps(pipe_num)

In [59]:
directory = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed/'
files = os.listdir(directory)

for f in files:
    #n = f.c
    os.rename(f, "out_ignore.txt")

files

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'ignore.txt' -> 'out_ignore.txt'

### Источники
1 - https://habr.com/ru/companies/yandex_praktikum/articles/756474/